In [25]:
import SimpleITK as sitk
import numpy as np
import trimesh
from tqdm import tqdm
from scipy.ndimage import gaussian_filter
from sklearn.manifold import Isomap
from pathlib import Path
from dataloaders import VIA11_Corrected_CS_Loader
import scipy.ndimage as ndimage
import open3d as o3d
import copy

import subprocess as sp


In [2]:
data_path = Path('/mrhome/vladyslavz/git/central-sulcus-analysis/shape_features/data/nobackup')


In [3]:
cs_ds = VIA11_Corrected_CS_Loader(bv_good=True, corrected=True, all_bv=False, preload=True)
        # unravell all the sulci
sulci_list = cs_ds.get_subjects()

100%|██████████| 286/286 [00:25<00:00, 11.25it/s]


In [4]:
idx = 13
print(f'Average sulcus automatically selected is {sulci_list[idx][0]}_{sulci_list[idx][1]}.')

Average sulcus automatically selected is right_sub-via033.


In [5]:
average_sulcus = sulci_list[idx]
average_sulcus

['right',
 'sub-via033',
 'corrected',
 <SimpleITK.SimpleITK.Image; proxy of <Swig Object of type 'std::vector< itk::simple::Image >::value_type *' at 0x7f8274252160> >]

In [6]:
sub5_left = sulci_list[4]
sub5_left

['left',
 'sub-via005',
 'corrected',
 <SimpleITK.SimpleITK.Image; proxy of <Swig Object of type 'std::vector< itk::simple::Image >::value_type *' at 0x7f82741f7990> >]

In [7]:
sulc2 = [x for x in sub5_left]

# need to flip if different hemispheres
if average_sulcus[0] != sulc2[0]:
    sulc2[3] = sitk.Flip(sulc2[3], [False, False, True])
    print('Flipped')

# register the sulci with ICP
s1_points = np.stack(np.where(sitk.GetArrayFromImage(average_sulcus[3]))).T
s2_points = np.stack(np.where(sitk.GetArrayFromImage(sulc2[3]))).T

# finds the transformation matrix sending a to b
trnsh_mat, transformed, __ = trimesh.registration.icp(a=s2_points,
                                                      b=s1_points,
                                                      max_iterations=1000,
                                                      scale=False,)

transformed_img = np.zeros_like(sitk.GetArrayFromImage(sulc2[3]))
transformed_img[tuple(np.round(transformed).astype(int).T)] = 1
transformed_img = sitk.GetImageFromArray(transformed_img.astype(np.float32))

res_sulci = [x for x in sulc2]
res_sulci[3] = transformed_img

Flipped


In [65]:
transformed 

array([[ 96.91800917, 205.01635312, 169.98567809],
       [ 96.94449797, 206.00826007, 169.86150504],
       [ 97.06413967, 206.87404163, 168.74970103],
       ...,
       [165.19776043, 153.99143718, 131.30320762],
       [165.22424923, 154.98334413, 131.17903457],
       [165.25073802, 155.97525108, 131.05486153]])

In [56]:

def fill_image_with_points(points, image_shape=(192, 288, 288)):
    # Create a grid representing the 3D space
    x, y, z = np.meshgrid(np.arange(image_shape[0]), np.arange(image_shape[1]), np.arange(image_shape[2]))
    grid = np.stack((x, y, z), axis=-1)

    # Interpolate the points using bilinear interpolation
    interpolated_values = ndimage.map_coordinates(grid,
                                                  points, order=1, mode='nearest')

    # Create the output image and fill it with 1s at the interpolated point coordinates
    output_image = np.zeros(image_shape)
    indices = np.round(interpolated_values).astype(int)
    mask = (indices >= 0) & (indices < image_shape[0])
    output_image[tuple(indices[mask].T)] = 1

    return output_image

# Example usage with the given list of point coordinates
point_coordinates = transformed.copy()

output_image = fill_image_with_points(point_coordinates)

RuntimeError: invalid shape for coordinate array

In [77]:
from scipy.interpolate import RegularGridInterpolator

interp = RegularGridInterpolator((np.arange(192), np.arange(288), np.arange(288)), output_image)

ValueError: The points in dimension 0 must be strictly ascending or descending

In [52]:
np.zeros_like(sitk.GetArrayFromImage(sulc2[3]))

array([[ 96.91800917, 205.01635312, 169.98567809],
       [ 96.94449797, 206.00826007, 169.86150504],
       [ 97.06413967, 206.87404163, 168.74970103],
       ...,
       [165.19776043, 153.99143718, 131.30320762],
       [165.22424923, 154.98334413, 131.17903457],
       [165.25073802, 155.97525108, 131.05486153]])

In [8]:
sub5REGTOavg = res_sulci[3]

In [19]:
i = sitk.ReadImage('/mnt/projects/VIA_Vlad/nobackup/BrainVisa/CS_edited/CFIN/LSulci_sub-via010_default_session_best_edit_NHT.nii.gz')
sub5REGTOavg.CopyInformation(i)
sub5REGTOavg = sitk.Cast(sub5REGTOavg, sitk.sitkInt16)

In [20]:
sitk.WriteImage(sub5REGTOavg, '/mrhome/vladyslavz/git/central-sulcus-analysis/shape_features/data/nobackup/sub5REGTOavg.nii.gz')
sitk.WriteImage(average_sulcus[3], '/mrhome/vladyslavz/git/central-sulcus-analysis/shape_features/data/nobackup/sub17_right_orig.nii.gz')
sitk.WriteImage(sulc2[3], '/mrhome/vladyslavz/git/central-sulcus-analysis/shape_features/data/nobackup/sub5_left_flipped.nii.gz')
sitk.WriteImage(sub5_left[3], '/mrhome/vladyslavz/git/central-sulcus-analysis/shape_features/data/nobackup/sub5_left_orig.nii.gz')

In [22]:
cmd = ['/mrhome/vladyslavz/portable_apps/brainvisa4_5/bin/AimsMeshBrain', '-i', '/mrhome/vladyslavz/git/central-sulcus-analysis/shape_features/data/nobackup/sub5REGTOavg.nii.gz', '-o', '/mrhome/vladyslavz/git/central-sulcus-analysis/shape_features/data/nobackup/sub5REGTOavg.ply']
subprocess.run(cmd)

reading image      : done
resolutions        : (0.899994 mm,0.902778 mm,0.902771 mm)
reading slice      :                      1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 7 7 7 7 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8 8 9 9 9 9 9 9 9 9 9 91010101010101010101011111111111111111111121212121212121212121313131313131313131314141414141414141414151515151515151515151616161616161616161617171717171717171717181818181818181818181919191919191919191920202020202020202020212121212121212121212222222222222222222223232323232323232323242424242424242424242525252525252525252526262626262626262626272727272727272727272828282828282828288
getting interface  :         done 
processing mesh    : reduced neighborhoodextended neighborhoovertices            smoothing  iteration                                            1   1   1   1   1   1   1   1   1   1   2   2   2   2   2   2   2   2   2   2   3                       pass 1/3 : decimating      

CompletedProcess(args=['/mrhome/vladyslavz/portable_apps/brainvisa4_5/bin/AimsMeshBrain', '-i', '/mrhome/vladyslavz/git/central-sulcus-analysis/shape_features/data/nobackup/sub5REGTOavg.nii.gz', '-o', '/mrhome/vladyslavz/git/central-sulcus-analysis/shape_features/data/nobackup/sub5REGTOavg.ply'], returncode=0)